In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, torch_dtype=torch.float16, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
print(base_model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "textminr/mistral-7b-4bit-ner")

In [ ]:
eval_prompt= """### Question: In a comprehensive research paper titled "The Future of Renewable Energy: Trends and Predictions," published in 2025, an in-depth analysis of emerging technologies in the field of sustainable energy is presented. This detailed study examines the latest advancements in solar, wind, and geothermal energy, and evaluates their potential impact on global energy consumption patterns. The paper also explores the economic and environmental implications of transitioning to renewable energy sources, including the challenges and opportunities for developing countries. Despite the exhaustive nature of the research and its implications for future energy policies, the author's name is not mentioned in the document.\n### Answer:"""

In [ ]:
eval_prompt2 = """### Question: A new exhibition traces the social history of the nation through 60 pairs of shoes Richard Brooks Sun 29 Oct 2023 13.00 CET You can always judge a man by his shoes, or so the old adage goes. And the same goes for woman, as shown in an exhibition in Winchester which promises to reveal the use and role of all sorts of footwear through the ages – and what it conveyed about the wearer’s status. Co-curator Tara McKinney Marinus says: “It’s a story of our social history, rather than footwear as fashion\n### Answer:"""

In [ ]:
eval_prompt3 = """### Question:Maritime Explorations details seafaring advancements in the 1800s.\n### Answer:"""

In [ ]:
# eval_prompt = "### Question: In their groundbreaking 2020 study on climate change impacts, Smith and Jones (2020) highlight the urgent need for sustainable practices.\n### Answer:"
model_input = tokenizer(eval_prompt3, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=1000, repetition_penalty=1.15)[0], skip_special_tokens=True))